# 1. 索引构建组件

### 1. extract_triples_eng_DEMO.py

In [ ]:
'''
extract_triples_eng_DEMO.py

数据集.json_文件 ->
取出answer,question,context -> answer_list,question_list,docs_list
doc -> 长文本sentence -> 输入张量model_inputs 
(仅用id ??? ) -> 输入张量的id列表 ids_list -> 对齐的id张量 ids_tensor
-> 使用Qwen_LLM -> generated_ids -> 解码为知识图谱三元组 response[]
人工整理为文件 -> 三元组文本行_文件
'''

device = "cuda:0" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen1.5-32B-Chat-AWQ",
    device_map=device
)
tokenizer = AutoTokenizer.from_pretrained("Qwen1.5-32B-Chat-AWQ")

path = './src/dataset/HOTPOPQA'

answer_list = []
question_list = []
docs_list = []
cnt = 0

with open(f'{path}/hotpot_dev_fullwiki_v1.json', 'r') as file:
    data = json.load(file)

    for i in range(len(data)):
        answer = data[i]['answer']
        question = data[i]['question']
        docs = data[i]['context']

        tmpd = []
        for j in range(len(docs)):
            tmpd.append(docs[j][1])

        answer_list.append(answer)
        question_list.append(question)
        docs_list.append(tmpd)


for docs in docs_list:
    ids_list = []
    cnt += 1
    if cnt > 186:   # 只处理第186条以后的数据
        for i in range(len(docs)):# 打包输入
            sentence = ''.join(docs[i][j] for j in range(len(docs[i])))# 将文档中的句子连接成完整文本

            messages = [{"role": "system",
                              "content": "You are an NLP assistant. Given a piece of text, you need to analyze its semantic information and generate a knowledge graph. Your output consists only of triples, considering only the text content, and avoiding newline characters. For example, (entity; relationship; entity),(entity; relationship; entity). The knowledge graph should be comprehensive, covering all information in the text."},
                        {"role": "user",
                              "content": 'Adam Collis is an American filmmaker and actor. He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010. He also studied cinema at the University of Southern California from 1991 to 1997. Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995). In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percent".'},
                                  {"role": "assistant",
                              "content": '(Adam Collis; nationality; American),(Adam Collis; profession; filmmaker),(Adam Collis; profession; actor),(Adam Collis; education; Duke University),(Adam Collis; education; University of California, Los Angeles),(Adam Collis; education; University of Southern California),(Adam Collis; attended; Duke University),(Adam Collis; attended; University of California, Los Angeles),(Adam Collis; attended; University of Southern California),(Adam Collis; first work; assistant director),(Adam Collis; work; "Love in the Ruins"),("Love in the Ruins"; director; Scott Derrickson),(Adam Collis; work date; 1995),(Adam Collis; role; "Crankshaft"),("Hundred Percent"; director; Eric Koyanagi),(Adam Collis; work; "Hundred Percent"),(Adam Collis; work date; 1998)'},
                        {"role": "user",
                         "content": 'Tyler Bates (born June 5, 1965) is an American musician, music producer, and composer for films, television, and video games. Much of his work is in the action and horror film genres, with films like "Dawn of the Dead, 300, Sucker Punch," and "John Wick." He has collaborated with directors like Zack Snyder, Rob Zombie, Neil Marshall, William Friedkin, Scott Derrickson, and James Gunn. With Gunn, he has scored every one of the director\'s films; including "Guardians of the Galaxy", which became one of the highest grossing domestic movies of 2014, and its 2017 sequel. In addition, he is also the lead guitarist of the American rock band Marilyn Manson, and produced its albums "The Pale Emperor" and "Heaven Upside Down".'},
                        {"role": "assistant",
                         "content": '(Tyler Bates; birthdate; June 5, 1965),(Tyler Bates; nationality; American),(Tyler Bates; profession; musician),(Tyler Bates; profession; music producer),(Tyler Bates; profession; composer),(Tyler Bates; works in; films),(Tyler Bates; works in; television),(Tyler Bates; works in; video games),(Tyler Bates; specializes in; action and horror film genres),(Tyler Bates; notable films; "Dawn of the Dead"),(Tyler Bates; notable films; "300"),(Tyler Bates; notable films; "Sucker Punch"),(Tyler Bates; notable films; "John Wick"),(Tyler Bates; collaborations; Zack Snyder),(Tyler Bates; collaborations; Rob Zombie),(Tyler Bates; collaborations; Neil Marshall),(Tyler Bates; collaborations; William Friedkin),(Tyler Bates; collaborations; Scott Derrickson),(Tyler Bates; collaborations; James Gunn),(Tyler Bates; collaborations; James Gunn),(Tyler Bates; collaborations; James Gunn),(Tyler Bates; scored; "Guardians of the Galaxy"),("Guardians of the Galaxy"; release year; 2014),("Guardians of the Galaxy"; grossing; high),("Guardians of the Galaxy"; sequel; released in 2017),(Tyler Bates; lead guitarist; Marilyn Manson),(Marilyn Manson; music albums; "The Pale Emperor"),(Marilyn Manson; music albums; "Heaven Upside Down")'}
                        ]
            messages.append({"role": "user",
                            "content": sentence})

            text = tokenizer.apply_chat_template(  # 将messages输入
                messages,
                tokenize=False,
                add_generation_prompt=True  # 添加生成提示
            )
            # 将文本转换为模型输入张量
            model_inputs = tokenizer([text], return_tensors="pt").to(device)
            ids_list.append(model_inputs.input_ids.squeeze())   # 存储ID序列
        
        # 对所有id进行padding填充对齐
        max_length = max([x.shape[0] for x in ids_list])
        pad_id = tokenizer.pad_token_id     # 获取需填充的ID
        for i in range(len(ids_list)):
            pad_length = max_length - ids_list[i].shape[0]
            ids_list[i] = torch.concat([torch.tensor(pad_id, device=device).repeat(pad_length), ids_list[i]], 0)

        ids_tensor = torch.stack(ids_list) # 将ids_list列表转换为张量ids_tensor

        # 生成知识图谱三元组
        generated_ids = model.generate( # 输入ids_tensor, model输出结果generated_ids
            ids_tensor,
            max_new_tokens=2048 # 最大新生成的token数量
        )
        generated_ids = [   # generated_ids = [len(input_ids) -> output_ids] -> 使仅包含新生成部分的ID
            output_ids[len(input_ids):] for input_ids, output_ids in zip(ids_tensor, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)# 将生成的ID转换为文本

        for i in range(len(response)):
            print(response[i])
        print('------------------------------------------')

### 2. src/dataset/preprocess/preprocess_hotpop.py

In [ ]:

'''
preprocess_hotpop.py

'{path}/outputs(500).txt'三元组 -> 知识图谱_文件/图张量_文件      # 得到 实体级层
数据集_文件 -> 知识图谱_文件/问题张量_文件, 文档张量_文件           # 得到 文档级层
'''

path = '../HOTPOPQA'

nodeslist = []
edgeslist = []
deslist = []

'''
[1] 三元组文本行编码: 三元组文本行_文件 -> nodeslist,edgeslist

三元组文本行_文件 -> step_one(): 
for file: textualize_graph(i,line)
-> 三元组
分批存入 -> nodeslist,edgeslist, des_文件
'''
def textualize_graph(i,line):# 从文本行中提取三元组并构建图结构
    # 查找所有三元组(src;rel;dst) -> triples
    triples = re.findall(r'\((.*?)\)', line)
    nodes = {}
    edges = []
    des = {}
    # triples -> nodes,edges,des
    for tri in triples:
        tri = tri.split(';') # 按分号分割三元组
        if len(tri) != 3:# 移除错误的三元组（长度不为3）
            if len(tri) >= 3:
                tri[2] = ''.join(tri[2:len(tri)]) # 如果长度大于3，将多余部分合并到第三个元素
            if len(tri) < 3:
                continue # 如果长度小于3，跳过此三元组
        
        src, edge_attr, dst = tri[0].strip(),tri[1].strip(),tri[2].strip()# 提取src,rel,dst
        # 添加新节点scr,dst -> nodes{}
        if src not in nodes:
            nodes[src] = len(nodes)     # node值 = 加入次序, node位置 = src值
        if dst not in nodes:
            nodes[dst] = len(nodes)
        # 添加边信息 -> edges[]          # edge值 = {三元组}
        edges.append({'src': nodes[src], 'edge_attr': edge_attr.strip(), 'dst': nodes[dst], })
        # 添加描述 -> des{}
        # 描述结构：des[src_id,dst_id]=(src,rel,dst) 
        des[str(nodes[src])+','+ str(nodes[dst])] = '(' + src +',' + edge_attr + ',' + dst + ')'

    # 将节点和边信息转换为 Pandas DataFrame 格式
    nodes = pd.DataFrame(nodes.items(), columns=['node_attr', 'node_id'])
    edges = pd.DataFrame(edges)
    return nodes,edges,des

# 生成知识图谱和描述文件
def step_one():
    # 创建用于存储描述文件的目录
    os.makedirs(f'{path}/KG_QA_vRobert/des500_new', exist_ok=True)
    cnt = 1 # 计数器，用于des文件命名
    tmp_nodeslist = [] 
    tmp_edgeslist = [] 
    tmp_deslist = [] 
    # file处理
    with open(f'{path}/outputs(500).txt', 'r') as file:
        for index, line in enumerate(file):
                # 如果不是分隔符行
                if line != '------------------------------------------\n':
                    # 提取图结构和描述
                    nodes, edges,des = textualize_graph(index,line)
                    tmp_nodeslist.append(nodes)
                    tmp_edgeslist.append(edges)
                    tmp_deslist.append(des)
                else:
                    # 遇到分隔符，将临时列表添加到全局列表
                    nodeslist.append(tmp_nodeslist)
                    edgeslist.append(tmp_edgeslist)
                    # 将描述信息保存为 pickle 文件
                    with open(f'{path}/KG_QA_vRobert/des500_new/{cnt}.pkl', 'wb') as file:
                        pickle.dump(tmp_deslist, file)
                    cnt += 1 # 计数器递增
                    # 重置临时列表
                    tmp_nodeslist = []
                    tmp_edgeslist = []
                    tmp_deslist = []

'''
[2] 编码知识图谱: nodeslist,edgeslist -> 知识图谱_文件/图张量_文件

nodeslist,edgeslist -> step_two():
取出nodes,edges -.node_attr列名> node_des[],edge_des[]
-> 使用roberta_LLM:
    node_des -> 张量x
    edge_des -> 张量e
torch: [edges.src, edges.dst] -> 索引张量edge_index
-> data(节点特征x, 边索引edge_index, 边特征edge_attr=e, 节点数量num_nodes)
存入-> 知识图谱_文件/图张量_文件/{i}/{j}
'''
def step_two():
    model = SentenceTransformer('../../../all-roberta-large-v1',device='cuda:1')

    # 创建用于存储知识图谱和图数据的目录
    os.makedirs(f'{path}/KG_QA_vRobert', exist_ok=True)
    os.makedirs(f'{path}/KG_QA_vRobert/graph500_new', exist_ok=True)

    # 遍历nodeslist
    for i in range(len(nodeslist)):
        # 为每个知识图谱创建子目录
        os.makedirs(f'{path}/KG_QA_vRobert/graph500_new/{i}', exist_ok=True)
        for j in range(len(nodeslist[i])):  # 取出nodes,edges
            nodes = nodeslist[i][j]
            edges = edgeslist[i][j]
            if not len(nodes) or not len(edges):# 如果节点或边为空, 跳过
                continue

            # 提取节点和边的描述
            node_des = nodes.node_attr.tolist()
            edge_des = edges.edge_attr.tolist()

            # 使用模型编码节点和边的描述，并转换为 PyTorch 张量
            x = model.encode(node_des,convert_to_tensor = True)
            e = model.encode(edge_des,convert_to_tensor = True)
            # 构建边的索引张量
            edge_index = torch.LongTensor([edges.src, edges.dst])

            # 创建 PyTorch Geometric 的 Data 对象，包含节点特征、边索引、边特征和节点数量
            data = Data(x=x, edge_index=edge_index, edge_attr=e, num_nodes=len(nodes))

            # 保存图数据到文件
            torch.save(data, f'{path}/KG_QA_vRobert/graph500_new/{i}/{j}.pt')

'''
[3] 编码问题: 数据集_文件.question -> 问题张量_文件

数据集_文件 -> encode_question()
取出question
-> 使用roberta_LLM:
    question -> 张量question_emb
存入 -> 问题张量_文件/{i}
'''
def encode_question():
    model = SentenceTransformer('../../../all-roberta-large-v1')
    # 创建用于存储问题嵌入的目录
    os.makedirs(f'{path}/questions_emb_vRobert', exist_ok=True)

    # 定义 HOTPOPQA 开发集的数据路径
    data_path = '../HOTPOPQA/hotpot_dev_fullwiki_v1.json'
    # 加载 JSON 格式的数据
    with open(data_path, 'r') as file:
        data = json.load(file)

    # 遍历数据集中的每个问题
    for i in range(len(data)):
        question = data[i]["question"] # 提取问题文本
        # 编码问题文本并转换为 PyTorch 张量
        question_emb = model.encode(question, convert_to_tensor=True)
        # 保存问题嵌入到文件
        torch.save(question_emb, f'{path}/questions_emb_vRobert/{i}.pt')

'''
[3] 编码文档: 数据集_文件.context -> 知识图谱_文件/文档张量_文件

数据集_文件 -> encode_doc()
取出context -> docs
取前500个数据项 -> 使用roberta_LLM:
                    拼合过的passage -> 张量doc_emb
存入 ->
doc_emb -> doc_emb_list -> 知识图谱_文件/文档张量_文件/{500}
passage -> doc_text_list -> 知识图谱_文件/文档列表_二进制文件/{500}

'''
def encode_doc():
    model = SentenceTransformer('../../../all-roberta-large-v1',device='cuda:1')

    # 创建用于存储知识图谱、文档文本和文档嵌入的目录
    os.makedirs(f'{path}/KG_QA_vRobert', exist_ok=True)
    os.makedirs(f'{path}/KG_QA_vRobert/doc_text', exist_ok=True)
    os.makedirs(f'{path}/KG_QA_vRobert/doc_emb', exist_ok=True)

    # 定义 HOTPOPQA 开发集的数据路径
    data_path = '../HOTPOPQA/hotpot_dev_fullwiki_v1.json'
    # 加载 JSON 格式的数据
    with open(data_path, 'r') as file:
        data = json.load(file)

    doc_text_list = [] # 存储文档文本列表
    doc_emb_list = [] # 存储文档嵌入列表

    # 遍历前 500 个数据项（假设每个数据项包含多个文档）
    for i in range(500):
        docs = data[i]["context"] # 提取上下文文档
        for doc in docs:
            # 将文档中的段落拼接成一个完整的文本
            passage = ''.join(doc[1][j] for j in range(len(doc[1])))
            # 编码文档文本并转换为 PyTorch 张量
            doc_emb = model.encode(passage, convert_to_tensor=True)
            doc_emb_list.append(doc_emb)
            doc_text_list.append(passage)

    # 保存所有文档嵌入到一个文件中
    torch.save(torch.stack(doc_emb_list), f'{path}/KG_QA_vRobert/doc_emb/{500}.pt')
    # 保存所有文档文本到 pickle 文件中
    with open(f'{path}/KG_QA_vRobert/doc_text/{500}.pkl', 'wb') as file:
        pickle.dump(doc_text_list, file)

if __name__ == '__main__':
    step_one() # 生成知识图谱和描述
    print("--- step_one() DONE! ---")
    step_two() # 编码知识图谱
    print("--- step_two() DONE! ---")
    encode_question() # 编码问题
    print("--- encode_question() DONE! ---")
    encode_doc() # 编码文档
    print("--- encode_doc() DONE! ---")
    print("--- preprocess_hotpop.py DONE! ---")

### 3. run_demo_EX_hotpop_v0.py

In [ ]:
'''
加载过程:
模型，预处理数据，问题嵌入，知识图谱数据，文档嵌入
计算过程:    # ???自己跟自己找相似
v0.py --- retrieval_func_two_stage()
v1.py --- retrieval_func_with_att()
'''

def main(args):
    batch_size = 1
    n = 0 # 起始处理的数据索引
    break_p = XXX # 结束处理的数据索引，XXX是一个占位符，取前K条数据进行实验

    '''
    加载过程:
    模型，预处理数据，问题嵌入，知识图谱数据，文档嵌入
    '''
    # 加载模型
    args.llm_model_path = llm_model_path[args.llm_model_name]
    model = load_model[args.model_name](args=args)              
    # 加载数据
    test_dataset_path = './src/dataset/HOTPOPQA/hotpot_dev_fullwiki_v1.json'
    test_dataset = load_dataset['HOTPOPQA'](test_dataset_path,size1=n,size2=break_p)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, pin_memory=True, shuffle=False)

    # 加载问题嵌入
    questions_emb_list = []
    for i in range(break_p):
        question_emb = torch.load(f"./src/dataset/HOTPOPQA/questions_emb_vRobert/{i}.pt")
        questions_emb_list.append(question_emb)

    # 加载知识图谱数据
    KG_lists = []
    test_folder_path = "./src/dataset/HOTPOPQA/KG_QA_vRobert"
    for i in range(n,break_p):
        KG_path = f'{test_folder_path}/graph/{i}/'
        if os.path.exists(KG_path):
            for j in range(20):
                if os.path.exists(KG_path + f'{j}.pt'):
                    KG = torch.load(KG_path + f'{j}.pt')
                    KG_lists.append(KG)
                else:
                    break
        else: # 如果路径不存在，也尝试添加KG    # ???
            KG_lists.append(KG)

    # 加载文档嵌入张量
    doc_embed_tensor = torch.load(f"./src/dataset/HOTPOPQA/KG_QA_vRobert/doc_emb/{XXX}.pt")
    doc_embed_tensor = doc_embed_tensor[:len(KG_lists)] # 取前{len(KG_lists)}个

    '''
    计算过程    # ???自己跟自己找相似
    '''
    # 将文档嵌入作为子图列表张量
    SG_lists_tensor = doc_embed_tensor
    # 计算内积。内积越大，向量方向越接近
    inner_product = torch.matmul(SG_lists_tensor, SG_lists_tensor.T)
    # 计算L2范数。欧式距离
    l2_norm = torch.norm(SG_lists_tensor, p=2, dim=1, keepdim=True)
    # 归一化内积
    n_prizes = inner_product / (l2_norm * l2_norm.T)
    # 获取前3个最大值及索引(最相似的3个)
    topk_n_values, topk_n_indices = torch.topk(n_prizes, 3, largest=True)
    # 转置id，获取3个子图的边
    SG_edges = topk_n_indices.T

    # 加载描述信息des[]
    des = []
    for i in range(n,break_p):
        des_path = f'{test_folder_path}/des/' + f'{i+1}.pkl'
        f = open(des_path, 'rb')
        tmp_des = pickle.load(f)
        for j in range(len(tmp_des)):
            des.append(tmp_des[j])

    '''
    评估阶段
    v0.py --- retrieval_func_two_stage()
    v1.py --- retrieval_func_with_att()
    '''
    predictions = [] # 存储模型预测结果
    labels = [] # 存储真实标签

    current_time = datetime.datetime.now()
    print('Times:', current_time) # 打印当前时间
    for step, batch in enumerate(test_loader):
        if step * batch_size < n:
            continue

        with torch.no_grad():
            ids = batch['id'].tolist()
            questions_emb = [questions_emb_list[i] for i in ids]
            # 执行两阶段检索，获取额外知识、掩码和检索到的描述
            test_extra_knowledges, _, masks,retri_des = retrieval_func_two_stage(questions_emb, doc_embed_tensor, KG_lists, SG_edges,des,device=model.device(),topk_n=20,intervals=0)
            # 使用模型进行文本推理
            output = model.inference_text(batch,retri_des)
            for j in range(len(output['questions'])):
                predictions.append(output['pred'][j]) # 添加预测结果
                labels.append(output['answers'][j]) # 添加真实答案

    # 将预测结果保存到pkl文件
    with open('./answer_list/ours_hotpop.pkl', 'wb') as file2:
        pickle.dump(predictions,file2)

    current_time = datetime.datetime.now()
    print('Times:', current_time) # 再次打印当前时间

    # 计算精确匹配(Exact Match)率
    Truenums1 = 0
    for i in range(len(predictions)):
        if labels[i].lower() in predictions[i].lower():
            Truenums1 += 1
    print('EM: ',Truenums1/len(predictions))


if __name__ == "__main__":
    args = parse_args_llama()

    main(args)
    
    torch.cuda.empty_cache()                # 清空CUDA缓存
    torch.cuda.reset_max_memory_allocated() # 重置CUDA最大内存分配记录
    gc.collect()                            # 执行垃圾回收

# 2. 知识检索组件

### 1. src/retrieval.py

In [ ]:
'''
retrieval_func:
# for i in range(len(neighbors)):
#     for edge in neighbors[i]:
#         if int(edge[1]) not in selected_SG[i]:
#             selected_SG[i].append(int(edge[1]))

retrieval_func_two_stage:
#         for i in range(len(neighbors)):
#             for edge in neighbors[i]:
#                 for j in range(1,len(edge)):
#                     if int(edge[j]) not in selected_SG[i]:
#                         selected_SG[i].append(int(edge[j]))
+++
# if (values[0] + pedal[j] in selected_node or values[1] + pedal[j] in selected_node) and str(
#     int(values[0])) + ',' + str(int(values[1])) in target_des.keys():

def retrieval_func_with_att:
# if (values[0] + pedal[j] in selected_node or values[1] + pedal[j] in selected_node) and str(
#     int(values[0])) + ',' + str(int(values[1])) in target_des.keys():

retrieval_func_via_doc:
空？？？

'''
def retrieval_func(questions_emb,SG_lists,KG_lists,SG_edges,des,device,topk_n=7,topk_e=3,intervals=0.40):
    questions_emb = torch.stack(questions_emb).to(device)
    SG_lists = torch.stack(SG_lists).to(device)
    inner_product = torch.matmul(questions_emb, SG_lists.T)
    l2_norm1 = torch.norm(questions_emb, p=2, dim=1, keepdim=True)
    l2_norm2 = torch.norm(SG_lists, p=2, dim=1, keepdim=True)
    sg_prizes = inner_product / (l2_norm1 * l2_norm2.T)

    topk_sg_values, topk_sg_indices = torch.topk(sg_prizes, 7, largest=True)
    selected_SG = topk_sg_indices.cpu().tolist()

    neighbors = SG_edges.T[topk_sg_indices.cpu().numpy()]
    for i in range(len(neighbors)):
        for edge in neighbors[i]:
            if int(edge[1]) not in selected_SG[i]:
                selected_SG[i].append(int(edge[1]))

    ret_data = []
    ret_mask = []
    ret_des = []
    for i in range(len(selected_SG)):
        KGs_x = [KG_lists[j].x for j in selected_SG[i]]
        KGs_edge_attr = [KG_lists[j].edge_attr for j in selected_SG[i]]
        KGs_edge_index = [KG_lists[j].edge_index for j in selected_SG[i]]
        KGs_numnodes = [KG_lists[j].num_nodes for j in selected_SG[i]]

        inner_product = torch.matmul(questions_emb[i], torch.cat(KGs_x, dim=0).T.to(device))
        l2_norm1 = torch.norm(questions_emb[i], p=2, dim=0, keepdim=True)
        l2_norm2 = torch.norm(torch.cat(KGs_x, dim=0).to(device), p=2, dim=1, keepdim=True).squeeze()
        n_prizes = inner_product / (l2_norm1 * l2_norm2)
        topk_n = min(len(n_prizes), topk_n)
        topk_n_values, topk_n_indices = torch.topk(n_prizes,topk_n , largest=True)
        selected_node = []
        for j in range(len(topk_n_indices)):
            if topk_n_values[j] > intervals:
                selected_node.append(topk_n_indices[j])

        pedal = [0]
        for j in range(len(KGs_numnodes)-1):
            pedal.append(pedal[-1]+KGs_numnodes[j])

        e = []
        e_attr = []
        tmp_des = []
        for j in range(len(KGs_edge_index)):
            target_des = des[int(selected_SG[i][j])]
            if not target_des:
                target_des = des[int(selected_SG[i][j])-1]
            for key,values in enumerate(KGs_edge_index[j].T):
                if (values[0] + pedal[j] in selected_node or values[1] + pedal[j] in selected_node) and str(
                    int(values[0]))+','+ str(int(values[1])) in target_des.keys():
                    e.append(values)
                    e_attr.append(KGs_edge_attr[j][key])
                    tmp_des.append(target_des[str(int(values[0]))+','+ str(int(values[1]))])
        ret_des.append(','.join(tmp_des))

    return ret_data, selected_SG,ret_mask,ret_des
 
def retrieval_func_two_stage(questions_emb,doc_embed_tensor,KG_lists,SG_edges,des,device,topk_n=7,topk_e=3,intervals=0.40):
    questions_emb = torch.stack(questions_emb).to(device)
    SG_lists = doc_embed_tensor.to(device)
    inner_product = torch.matmul(questions_emb, SG_lists.T)
    l2_norm1 = torch.norm(questions_emb, p=2, dim=1, keepdim=True)
    l2_norm2 = torch.norm(SG_lists, p=2, dim=1, keepdim=True)
    sg_prizes = inner_product / (l2_norm1 * l2_norm2.T)

    topk_sg_values, topk_sg_indices = torch.topk(sg_prizes, 3, largest=True)
    selected_SG = topk_sg_indices.cpu().tolist()

    '''
    neighbors = SG_edges.T[topk_sg_indices.cpu().numpy()]
        for i in range(len(neighbors)):
            for edge in neighbors[i]:
                for j in range(1,len(edge)):
                    if int(edge[j]) not in selected_SG[i]:
                        selected_SG[i].append(int(edge[j]))
    '''
    neighbors = SG_edges.T[topk_sg_indices.cpu().numpy()]
    for i in range(len(neighbors)):
        for edge in neighbors[i]:
            for j in range(1,len(edge)):
                if int(edge[j]) not in selected_SG[i]:
                    selected_SG[i].append(int(edge[j]))

    ret_data = []
    ret_mask = []
    ret_des = []
    for i in range(len(selected_SG)):
        KGs_x = [KG_lists[j].x for j in selected_SG[i]]
        KGs_edge_attr = [KG_lists[j].edge_attr for j in selected_SG[i]]
        KGs_edge_index = [KG_lists[j].edge_index for j in selected_SG[i]]
        KGs_numnodes = [KG_lists[j].num_nodes for j in selected_SG[i]]

        inner_product = torch.matmul(questions_emb[i], torch.cat(KGs_x, dim=0).T.to(device))
        l2_norm1 = torch.norm(questions_emb[i], p=2, dim=0, keepdim=True)
        l2_norm2 = torch.norm(torch.cat(KGs_x, dim=0).to(device), p=2, dim=1, keepdim=True).squeeze()
        n_prizes = inner_product / (l2_norm1 * l2_norm2)
        topk_n = min(len(n_prizes), topk_n)
        topk_n_values, topk_n_indices = torch.topk(n_prizes, topk_n, largest=True)
        selected_node = []
        for j in range(len(topk_n_indices)):
            if topk_n_values[j] > intervals:
                selected_node.append(topk_n_indices[j])

        pedal = [0]
        for j in range(len(KGs_numnodes) - 1):
            pedal.append(pedal[-1] + KGs_numnodes[j])

        e = []
        e_attr = []
        tmp_des = []
        for j in range(len(KGs_edge_index)):
            target_des = des[int(selected_SG[i][j])]
            if not target_des:
                target_des = des[int(selected_SG[i][j]) - 1]
            for key, values in enumerate(KGs_edge_index[j].T):
                '''
                IF selected_node AND target_des(selected_SG)
                '''
                if (values[0] + pedal[j] in selected_node or values[1] + pedal[j] in selected_node) and str(
                        int(values[0])) + ',' + str(int(values[1])) in target_des.keys():
                    e.append(values)
                    e_attr.append(KGs_edge_attr[j][key])
                    tmp_des.append(target_des[str(int(values[0])) + ',' + str(int(values[1]))])
        ret_des.append(','.join(tmp_des))

    return ret_data, selected_SG, ret_mask, ret_des

def retrieval_func_with_att(questions_emb,doc_embed_tensor,KG_lists,SG,des,device,topk_n=7,topk_e=3,intervals=0.40):
    questions_emb = torch.stack(questions_emb).to(device)
    SG_lists = doc_embed_tensor.to(device)
    inner_product = torch.matmul(questions_emb, SG_lists.T)
    l2_norm1 = torch.norm(questions_emb, p=2, dim=1, keepdim=True)
    l2_norm2 = torch.norm(SG_lists, p=2, dim=1, keepdim=True)
    sg_prizes = inner_product / (l2_norm1 * l2_norm2.T)

    k = 4
    topk_sg_values, topk_sg_indices = torch.topk(sg_prizes, k, largest=True)
    selected_SG = topk_sg_indices.cpu().tolist()
    weight_list = [1]*k

    new_node = []
    for node in selected_SG[0]:
        for neib in list(nx.neighbors(SG, node)):  # find 1_th neighbors
            if neib not in selected_SG[0] and neib not in new_node:
                new_node.append(neib)
                weight_list.append(SG[node][neib]['weight'])
    new_new_node = new_node.copy()

    selected_SG[0] += new_new_node

    ret_data = []
    ret_mask = []
    ret_des = []
    for i in range(len(selected_SG)):
        KGs_x = [KG_lists[j].x for j in selected_SG[i]]
        KGs_edge_attr = [KG_lists[j].edge_attr for j in selected_SG[i]]
        KGs_edge_index = [KG_lists[j].edge_index for j in selected_SG[i]]
        KGs_numnodes = [KG_lists[j].num_nodes for j in selected_SG[i]]

        attention = []
        for j in range(len(weight_list)):
            attention += [weight_list[j]] * KGs_numnodes[j]

        inner_product = torch.matmul(questions_emb[i], torch.cat(KGs_x, dim=0).T.to(device))
        l2_norm1 = torch.norm(questions_emb[i], p=2, dim=0, keepdim=True)
        l2_norm2 = torch.norm(torch.cat(KGs_x, dim=0).to(device), p=2, dim=1, keepdim=True).squeeze()
        n_prizes = inner_product / (l2_norm1 * l2_norm2)
        topk_n = min(len(n_prizes), topk_n)
        topk_n_values, topk_n_indices = torch.topk(torch.stack([n_prizes[j] * attention[j] for j in range(len(attention))]), topk_n, largest=True)
        selected_node = []
        for j in range(len(topk_n_indices)):
            if topk_n_values[j] > intervals:
                selected_node.append(topk_n_indices[j])

        pedal = [0]
        for j in range(len(KGs_numnodes) - 1):
            pedal.append(pedal[-1] + KGs_numnodes[j])

        e = []
        e_attr = []
        tmp_des = []
        for j in range(len(KGs_edge_index)):
            target_des = des[int(selected_SG[i][j])]
            if not target_des:
                target_des = des[int(selected_SG[i][j]) - 1]
            for key, values in enumerate(KGs_edge_index[j].T):
                if (values[0] + pedal[j] in selected_node or values[1] + pedal[j] in selected_node) and str(
                        int(values[0])) + ',' + str(int(values[1])) in target_des.keys():
                    e.append(values)
                    e_attr.append(KGs_edge_attr[j][key])
                    tmp_des.append(target_des[str(int(values[0])) + ',' + str(int(values[1]))])
        ret_des.append(','.join(tmp_des))

    return ret_data, selected_SG, ret_mask, ret_des

# 这似乎未完成 ???
def retrieval_func_via_doc(questions_emb,doc_embed_list,doc_text_list,device,topk_n=7,topk_e=3,intervals=0.44,SG_edges=None):
    # questions_emb 和 graph_emb之间有gap啊
    questions_emb = torch.stack(questions_emb).to(device)
    # doc_embed = torch.stack(doc_embed_list).to(device)
    doc_embed = doc_embed_list.to(device)
    inner_product = torch.matmul(questions_emb, doc_embed.T)
    l2_norm1 = torch.norm(questions_emb, p=2, dim=1, keepdim=True)
    l2_norm2 = torch.norm(doc_embed, p=2, dim=1, keepdim=True)
    sg_prizes = inner_product / (l2_norm1 * l2_norm2.T)

    topk_sg_values, topk_sg_indices = torch.topk(sg_prizes, 1, largest=True)

    selected_SG = topk_sg_indices.cpu().tolist()

    ret_data = []
    ret_mask = []
    ret_des = []
    for i in range(len(selected_SG)):
        e = []
        e_attr = []
        tmp_des = []
        for j in range(len(selected_SG[i])):
            target_des = doc_text_list[int(selected_SG[i][j])]
            tmp_des.append(target_des)
        ret_des.append(','.join(tmp_des))

    return ret_data, selected_SG,ret_mask,ret_des